Preprocessing Menggunakan Spacy
===

***Margareta Valencia (A11.2022.14704)***

---

## memakai NLTK

In [5]:
# token
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Acer\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
def word_tokenize_wrapper(text):
 return word_tokenize(text)

In [7]:
teks_nltk = word_tokenize_wrapper('Pagi yang cerah memancarkan sinarnya melalui jendela, memberikan semangat baru untuk memulai hari dengan penuh energi.')

In [8]:
teks_nltk

['Pagi',
 'yang',
 'cerah',
 'memancarkan',
 'sinarnya',
 'melalui',
 'jendela',
 ',',
 'memberikan',
 'semangat',
 'baru',
 'untuk',
 'memulai',
 'hari',
 'dengan',
 'penuh',
 'energi',
 '.']

## memakai spacy

In [13]:
!pip install spacy

     ---------------------------------------- 0.0/103.4 kB ? eta -:--:--
     ---------------------------------------- 0.0/103.4 kB ? eta -:--:--
     ---------------------------------------- 0.0/103.4 kB ? eta -:--:--
     ---------------------------------------- 0.0/103.4 kB ? eta -:--:--
     --- ------------------------------------ 10.2/103.4 kB ? eta -:--:--
     --- ------------------------------------ 10.2/103.4 kB ? eta -:--:--
     --- ------------------------------------ 10.2/103.4 kB ? eta -:--:--
     --- ------------------------------------ 10.2/103.4 kB ? eta -:--:--
     --- ------------------------------------ 10.2/103.4 kB ? eta -:--:--
     --- ------------------------------------ 10.2/103.4 kB ? eta -:--:--
     --- ------------------------------------ 10.2/103.4 kB ? eta -:--:--
     --- ------------------------------------ 10.2/103.4 kB ? eta -:--:--
     --------------- ---------------------- 41.0/103.4 kB 89.6 kB/s eta 0:00:01
     ----------------------------- -

In [15]:
from spacy.lang.id import Indonesian
import spacy

nlp = Indonesian()  # use directly
nlp = spacy.blank('id')  # blank instance'
Teks = nlp('Pagi yang cerah memancarkan sinarnya melalui jendela, memberikan semangat baru untuk memulai hari dengan penuh energi.')

In [16]:
Token_kata = [token.text for token in Teks]

In [17]:
Token_kata

['Pagi',
 'yang',
 'cerah',
 'memancarkan',
 'sinarnya',
 'melalui',
 'jendela',
 ',',
 'memberikan',
 'semangat',
 'baru',
 'untuk',
 'memulai',
 'hari',
 'dengan',
 'penuh',
 'energi',
 '.']

# load library

In [18]:
import re
import string
import time
from copy import deepcopy

# load dataset

dataset ini menggunakan sentimen dan emosi.

kali ini kita akan menggunakan sentimen terlebih dahulu

silakan drop kolom Emosi

In [19]:
import pandas as pd

In [22]:
df = pd.read_csv('Data_Twitter.csv', delimiter='\t')

In [23]:
df.head()

,Date,User,Tweet,sentiment
0,2022-03-31 14:32:04+00:00,pikobar_jabar,Ketahui informasi pembagian #PPKM di wilayah J...,1
1,2022-03-31 09:26:00+00:00,inewsdotid,Tempat Ibadah di Wilayah PPKM Level 1 Boleh Be...,1
2,2022-03-31 05:02:34+00:00,vdvc_talk,"Juru bicara Satgas Covid-19, Wiku Adisasmito m...",1
3,2022-03-30 14:23:10+00:00,pikobar_jabar,Ketahui informasi pembagian #PPKM di wilayah J...,1
4,2022-03-30 11:28:57+00:00,tvOneNews,Kementerian Agama menerbitkan Surat Edaran Nom...,1


In [25]:
df = df.drop(['Date'], axis=1)

In [26]:
df

,User,Tweet,sentiment
0,pikobar_jabar,Ketahui informasi pembagian #PPKM di wilayah J...,1
1,inewsdotid,Tempat Ibadah di Wilayah PPKM Level 1 Boleh Be...,1
2,vdvc_talk,"Juru bicara Satgas Covid-19, Wiku Adisasmito m...",1
3,pikobar_jabar,Ketahui informasi pembagian #PPKM di wilayah J...,1
4,tvOneNews,Kementerian Agama menerbitkan Surat Edaran Nom...,1
...,...,...,...
23639,bananabluff,noelle loses a bet to akarsha and it somehow e...,1
23640,Auqroix,they call her... weekeeshee...\n#butterflysoup...,1
23641,Auqroix,"put out what you wanna see more of, amirite ga...",1
23642,Auqroix,"i don't need anybody, i'm fine here on my own\...",1


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23644 entries, 0 to 23643
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   User       23644 non-null  object
 1   Tweet      23644 non-null  object
 2   sentiment  23644 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 554.3+ KB


# Preprocessing dataset
kita akan pakai modul dari AMS 01-03

disamping itu kita akan memakai modul  https://github.com/cbaziotis/ekphrasis 

untuk instalasi, silakan buka tanda pagar dan jalankan

In [28]:
!pip install ekphrasis

In [29]:
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['email', 'percent', 'money', 'phone', 'user',
        'time', 'date', 'number'],
    # terms that will be annotated
    #annotate={"hashtag", "allcaps", "elongated", "repeated",'emphasis', 'censored'},
    annotate={"hashtag"},
    fix_html=True,  # fix HTML tokens
    
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter", 
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=False,  # spell correction for elongated words
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)


C:\Users\Acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\ekphrasis\classes\tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Reading twitter - 1grams ...
Reading twitter - 2grams ...
Reading twitter - 1grams ...


C:\Users\Acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\ekphrasis\classes\exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


In [30]:
# panggil ekphrasis

def bersih_data(text):
    return " ".join(text_processor.pre_process_doc(text))

In [31]:
# fungsi dari AMS 01-03. silakan cek bagaimana saya merubah menjadi fungsi

def non_ascii(text):
    return text.encode('ascii', 'replace').decode('ascii')

def remove_space_alzami(text):
    return " ".join(text.split())

def remove_emoji_alzami(text):
    return ' '.join(re.sub("([x#][A-Za-z0-9]+)"," ", text).split())

def remove_tab(text):
    return text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")

def remove_tab2(text):
    return re.sub('\s+',' ',text)

def remove_rt(text):
    return text.replace('rt'," ")

def remove_mention(text):
    return ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())

def remove_incomplete_url(text):
    return text.replace("http://", " ").replace("https://", " ")

def remove_single_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

def remove_excessive_dot(text):
    return text.replace('..'," ")

def change_stripe(text):
    return text.replace('-'," ")

def lower(text):
    return text.lower()

def remove_single_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

def remove_excessive_dot(text):
    return text.replace('..'," ")

def lower(text):
    return text.lower()

def remove_whitespace_LT(text):
    return text.strip()

def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

def remove_punctuation(text):
    remove = string.punctuation
    remove = remove.replace("_", "") # don't remove hyphens
    pattern = r"[{}]".format(remove) # create the pattern
    return re.sub(pattern, "", text) 

In [32]:
# hapus untuk <>
def remove_number_eks(text):
    return text.replace('<number>'," ")

def remove_angka(text):
    return re.sub(r"\d+", "", text) 

def remove_URL_eks(text):
    return text.replace('URL'," ").replace('url'," ")

def space_punctuation(text):
    return re.sub('(?<! )(?=[.,!?()])|(?<=[.,!?()])(?! )', r' ', text)

## lakukan pembersihan dengan memanggil fungsi yang didefinisikan diatas

In [33]:
i = 0
final_string = []
s = ""
for text in df['Tweet'].values:
    filteredSentence = []
    EachReviewText = ""
    proc = lower(text)
    proc = change_stripe(text)
    proc = remove_emoji_alzami(proc)
    proc = remove_tab(proc)
    proc = remove_tab2(proc)
    proc = non_ascii(proc)
    proc = remove_incomplete_url(proc)
    proc = remove_excessive_dot(proc)
    proc = remove_whitespace_LT(proc)
    proc = remove_whitespace_multiple(proc)
    proc = remove_single_char(proc)
    proc = space_punctuation(proc)
    proc = remove_punctuation(proc)
    proc = remove_space_alzami(proc)
    proc = bersih_data(proc)
    #proc = remove_rt(proc)
    proc = remove_number_eks(proc)
    proc = remove_angka(proc) 
    proc = remove_URL_eks(proc)
    EachReviewText = proc
    final_string.append(EachReviewText)

In [34]:
df["step01"] = final_string

In [36]:
df.head(10)

,User,Tweet,sentiment,step01
0,pikobar_jabar,Ketahui informasi pembagian #PPKM di wilayah J...,1,ketahui informasi pembagian di wilayah jabar b...
1,inewsdotid,Tempat Ibadah di Wilayah PPKM Level 1 Boleh Be...,1,tempat ibadah di wilayah ppkm level boleh be...
2,vdvc_talk,"Juru bicara Satgas Covid-19, Wiku Adisasmito m...",1,juru bicara satgas covid wiku adisasmito men...
3,pikobar_jabar,Ketahui informasi pembagian #PPKM di wilayah J...,1,ketahui informasi pembagian di wilayah jabar b...
4,tvOneNews,Kementerian Agama menerbitkan Surat Edaran Nom...,1,kementerian agama menerbitkan surat edaran nom...
5,lampungpost_,"Kapasitas tempat ibadah, termasuk masjid, yang...",1,kapasitas tempat ibadah termasuk masjid yang b...
6,akusehatklinik,"Halo Sobat Sehat\n\nDengan adanya kondisi ini,...",1,halo sobat sehat dengan adanya kondisi ini pem...
7,matamilenialID,Mitigasi Penting untuk Cegah Penyebaran Varian...,1,mitigasi penting untuk cegah penyebaran varian...
8,lampungpostid,Sebanyak 5 (lima) kabupaten di Provinsi Lampun...,1,sebanyak lima kabupaten di provinsi lampung ...
9,gemaposID,Komentar Satgas Penanganan Covid-19 Tentang Bu...,1,komentar satgas penanganan covid tentang buk...


## hapus data kosong

kadang ada data yang kosong. dan ini tidak bisa kita apa-apakan, maka dari itu kita hapus saja

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23644 entries, 0 to 23643
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   User       23644 non-null  object
 1   Tweet      23644 non-null  object
 2   sentiment  23644 non-null  int64 
 3   step01     23644 non-null  object
dtypes: int64(1), object(3)
memory usage: 739.0+ KB


In [38]:
df_hapus = df[~df['step01'].str.contains(" ")]

In [39]:
df_hapus.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1163 entries, 154 to 23637
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   User       1163 non-null   object
 1   Tweet      1163 non-null   object
 2   sentiment  1163 non-null   int64 
 3   step01     1163 non-null   object
dtypes: int64(1), object(3)
memory usage: 45.4+ KB


In [40]:
df_hapus.head(10)

,User,Tweet,sentiment,step01
154,mediaindopos,https://t.co/jzZ3PFEPhT\n\n#ppkm\n#jawabali,1,cojzzpfepht
248,salsaaii,https://t.co/zcr45lhM6h #ppkm,1,cozcrlhmh
249,salsaaii,https://t.co/SHQ8ANOOyW #ppkm,1,coshqanooyw
286,Macardianto,#ppkm\nhttps://t.co/xpECW4SKKi,1,co
350,inosukesama17,#PPKMTEKANCOVID #ppkm https://t.co/vo94ekSjZd,1,covoeksjzd
351,inosukesama17,#PPKMTEKANCOVID #ppkm https://t.co/tlxVCOw8Mj,1,cotl
352,inosukesama17,#PPKMTEKANCOVID #ppkm https://t.co/inCCRKtRWr,1,coinccrktrwr
353,inosukesama17,#PPKMTEKANCOVID #ppkm https://t.co/3mhnnp2L2S,1,comhnnpls
354,inosukesama17,#PPKMTEKANCOVID #ppkm https://t.co/RxuQb2bMqy,1,co
355,inosukesama17,#PPKMTEKANCOVID #ppkm https://t.co/EajafS0aos,1,coeajafsaos


In [41]:
df_new = df[~df.isin(df_hapus)].dropna()

In [42]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22481 entries, 0 to 23643
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   User       22481 non-null  object 
 1   Tweet      22481 non-null  object 
 2   sentiment  22481 non-null  float64
 3   step01     22481 non-null  object 
dtypes: float64(1), object(3)
memory usage: 878.2+ KB


In [43]:
df_new

,User,Tweet,sentiment,step01
0,pikobar_jabar,Ketahui informasi pembagian #PPKM di wilayah J...,1.0,ketahui informasi pembagian di wilayah jabar b...
1,inewsdotid,Tempat Ibadah di Wilayah PPKM Level 1 Boleh Be...,1.0,tempat ibadah di wilayah ppkm level boleh be...
2,vdvc_talk,"Juru bicara Satgas Covid-19, Wiku Adisasmito m...",1.0,juru bicara satgas covid wiku adisasmito men...
3,pikobar_jabar,Ketahui informasi pembagian #PPKM di wilayah J...,1.0,ketahui informasi pembagian di wilayah jabar b...
4,tvOneNews,Kementerian Agama menerbitkan Surat Edaran Nom...,1.0,kementerian agama menerbitkan surat edaran nom...
...,...,...,...,...
23639,bananabluff,noelle loses a bet to akarsha and it somehow e...,1.0,noelle loses bet to akarsha and it somehow end...
23640,Auqroix,they call her... weekeeshee...\n#butterflysoup...,1.0,they call her weekeeshee cocycpjwvef
23641,Auqroix,"put out what you wanna see more of, amirite ga...",1.0,put out what you wanna see more of amirite gay...
23642,Auqroix,"i don't need anybody, i'm fine here on my own\...",1.0,don need anybody fine here on my own but if ne...


## normalisasi kata slang menjadi kata baku

In [44]:
# token
import nltk
from nltk.tokenize import word_tokenize 

In [45]:
def word_tokenize_wrapper(text):
  return word_tokenize(text)

In [46]:
df_new['tokens'] = df['step01'].apply(word_tokenize_wrapper)

In [47]:
df_new.head(10)

,User,Tweet,sentiment,step01,tokens
0,pikobar_jabar,Ketahui informasi pembagian #PPKM di wilayah J...,1.0,ketahui informasi pembagian di wilayah jabar b...,"[ketahui, informasi, pembagian, di, wilayah, j..."
1,inewsdotid,Tempat Ibadah di Wilayah PPKM Level 1 Boleh Be...,1.0,tempat ibadah di wilayah ppkm level boleh be...,"[tempat, ibadah, di, wilayah, ppkm, level, bol..."
2,vdvc_talk,"Juru bicara Satgas Covid-19, Wiku Adisasmito m...",1.0,juru bicara satgas covid wiku adisasmito men...,"[juru, bicara, satgas, covid, wiku, adisasmito..."
3,pikobar_jabar,Ketahui informasi pembagian #PPKM di wilayah J...,1.0,ketahui informasi pembagian di wilayah jabar b...,"[ketahui, informasi, pembagian, di, wilayah, j..."
4,tvOneNews,Kementerian Agama menerbitkan Surat Edaran Nom...,1.0,kementerian agama menerbitkan surat edaran nom...,"[kementerian, agama, menerbitkan, surat, edara..."
5,lampungpost_,"Kapasitas tempat ibadah, termasuk masjid, yang...",1.0,kapasitas tempat ibadah termasuk masjid yang b...,"[kapasitas, tempat, ibadah, termasuk, masjid, ..."
6,akusehatklinik,"Halo Sobat Sehat\n\nDengan adanya kondisi ini,...",1.0,halo sobat sehat dengan adanya kondisi ini pem...,"[halo, sobat, sehat, dengan, adanya, kondisi, ..."
7,matamilenialID,Mitigasi Penting untuk Cegah Penyebaran Varian...,1.0,mitigasi penting untuk cegah penyebaran varian...,"[mitigasi, penting, untuk, cegah, penyebaran, ..."
8,lampungpostid,Sebanyak 5 (lima) kabupaten di Provinsi Lampun...,1.0,sebanyak lima kabupaten di provinsi lampung ...,"[sebanyak, lima, kabupaten, di, provinsi, lamp..."
9,gemaposID,Komentar Satgas Penanganan Covid-19 Tentang Bu...,1.0,komentar satgas penanganan covid tentang buk...,"[komentar, satgas, penanganan, covid, tentang,..."


In [48]:
normalized_word = pd.read_csv('kamus_clean.csv')

In [49]:
normalized_word_dict = {}

for index, row in normalized_word.iterrows():
    if row[0] not in normalized_word_dict:
        normalized_word_dict[row[0]] = row[1] 

def normalized_term(document):
    return [normalized_word_dict[term] if term in normalized_word_dict else term for term in document]

C:\Users\Acer\AppData\Local\Temp\ipykernel_93444\2408214049.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if row[0] not in normalized_word_dict:
C:\Users\Acer\AppData\Local\Temp\ipykernel_93444\2408214049.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  normalized_word_dict[row[0]] = row[1]


In [50]:
df_new['final_tokens'] = df_new['tokens'].apply(normalized_term)

In [51]:
i=0
final_string_tokens = []
for text in df_new['final_tokens'].values:
    EachReviewText = ""
    EachReviewText = ' '.join(text)
    final_string_tokens.append(EachReviewText)

In [52]:
df_new["step02"] = final_string_tokens

In [53]:
df_new.head(10)

,User,Tweet,sentiment,step01,tokens,final_tokens,step02
0,pikobar_jabar,Ketahui informasi pembagian #PPKM di wilayah J...,1.0,ketahui informasi pembagian di wilayah jabar b...,"[ketahui, informasi, pembagian, di, wilayah, j...","[ketahui, informasi, pembagian, di, wilayah, j...",ketahui informasi pembagian di wilayah jabar b...
1,inewsdotid,Tempat Ibadah di Wilayah PPKM Level 1 Boleh Be...,1.0,tempat ibadah di wilayah ppkm level boleh be...,"[tempat, ibadah, di, wilayah, ppkm, level, bol...","[tempat, ibadah, di, wilayah, ppkm, level, bol...",tempat ibadah di wilayah ppkm level boleh berk...
2,vdvc_talk,"Juru bicara Satgas Covid-19, Wiku Adisasmito m...",1.0,juru bicara satgas covid wiku adisasmito men...,"[juru, bicara, satgas, covid, wiku, adisasmito...","[juru, bicara, satgas, covid, wiku, adisasmito...",juru bicara satgas covid wiku adisasmito menje...
3,pikobar_jabar,Ketahui informasi pembagian #PPKM di wilayah J...,1.0,ketahui informasi pembagian di wilayah jabar b...,"[ketahui, informasi, pembagian, di, wilayah, j...","[ketahui, informasi, pembagian, di, wilayah, j...",ketahui informasi pembagian di wilayah jabar b...
4,tvOneNews,Kementerian Agama menerbitkan Surat Edaran Nom...,1.0,kementerian agama menerbitkan surat edaran nom...,"[kementerian, agama, menerbitkan, surat, edara...","[kementerian, agama, menerbitkan, surat, edara...",kementerian agama menerbitkan surat edaran nom...
5,lampungpost_,"Kapasitas tempat ibadah, termasuk masjid, yang...",1.0,kapasitas tempat ibadah termasuk masjid yang b...,"[kapasitas, tempat, ibadah, termasuk, masjid, ...","[kapasitas, tempat, ibadah, termasuk, masjid, ...",kapasitas tempat ibadah termasuk masjid yang b...
6,akusehatklinik,"Halo Sobat Sehat\n\nDengan adanya kondisi ini,...",1.0,halo sobat sehat dengan adanya kondisi ini pem...,"[halo, sobat, sehat, dengan, adanya, kondisi, ...","[halo, sobat, sehat, dengan, adanya, kondisi, ...",halo sobat sehat dengan adanya kondisi ini pem...
7,matamilenialID,Mitigasi Penting untuk Cegah Penyebaran Varian...,1.0,mitigasi penting untuk cegah penyebaran varian...,"[mitigasi, penting, untuk, cegah, penyebaran, ...","[mitigasi, penting, untuk, cegah, penyebaran, ...",mitigasi penting untuk cegah penyebaran varian...
8,lampungpostid,Sebanyak 5 (lima) kabupaten di Provinsi Lampun...,1.0,sebanyak lima kabupaten di provinsi lampung ...,"[sebanyak, lima, kabupaten, di, provinsi, lamp...","[sebanyak, lima, kabupaten, di, provinsi, lamp...",sebanyak lima kabupaten di provinsi lampung tu...
9,gemaposID,Komentar Satgas Penanganan Covid-19 Tentang Bu...,1.0,komentar satgas penanganan covid tentang buk...,"[komentar, satgas, penanganan, covid, tentang,...","[komentar, satgas, penanganan, covid, tentang,...",komentar satgas penanganan covid tentang buka ...


# simpan

In [54]:
df.to_csv('clean_dataset2.csv',sep=";")